In [ ]:
# 🧠 Sistema Fenomenológico Estructural v2.2

Este notebook procesa textos fenomenológicos y genera análisis detallados usando el sistema YO emergente.

In [ ]:
# Instalar dependencias necesarias
!pip install pandas numpy scikit-learn pyyaml

In [ ]:
import os

# Crear directorios principales
os.makedirs('configuracion', exist_ok=True)
os.makedirs('scripts', exist_ok=True)
os.makedirs('scripts/modelos', exist_ok=True)
os.makedirs('entrada_bruta', exist_ok=True)
os.makedirs('features_extraidas', exist_ok=True)
os.makedirs('logs_sistema', exist_ok=True)
os.makedirs('procesado/nodos_fenomenologicos/v1/fenomenos', exist_ok=True)
os.makedirs('procesado/nodos_fenomenologicos/v1/contextos', exist_ok=True)
os.makedirs('procesado/nodos_fenomenologicos/v1/macrocontextos', exist_ok=True)
os.makedirs('procesado/nodos_fenomenologicos/v1/redes', exist_ok=True)
os.makedirs('procesado/nodos_fenomenologicos/v1/conceptos', exist_ok=True)
os.makedirs('procesado/nodos_fenomenologicos/v1/metacampos', exist_ok=True)

print("Directorios creados.")

# --- Archivo: configuracion/config.yaml ---
config_content = """
procesamiento:
  idioma: spanish
  min_longitud_texto: 10
  max_features_tfidf: 5000
  ngram_range: [1, 2]
  eliminar_stopwords: true

clasificacion:
  algoritmo: RandomForest
  n_estimators: 100
  test_size: 0.5 # Ajustado para evitar errores con pocos datos
  random_state: 42

fenomenologia:
  categorias_base:
    - experiencia_vivida
    - conciencia_temporal
    - intersubjetividad
    - corporalidad
    - espacialidad
    - mundaneidad
    - afectividad
    - intencionalidad
  umbral_relevancia: 0.6
  generar_nodos_obsidian: true

sistema:
  backup_automatico: true
  intervalo_backup_horas: 24
  max_backups: 7
  generar_reportes: true

modelo_semantico:
  rutas:
    fenomenos: procesado/nodos_fenomenologicos/v1/fenomenos
    contextos: procesado/nodos_fenomenologicos/v1/contextos
    macrocontextos: procesado/nodos_fenomenologicos/v1/macrocontextos
    redes: procesado/nodos_fenomenologicos/v1/redes
    conceptos_emergentes: procesado/nodos_fenomenologicos/v1/conceptos
    metacampos: procesado/nodos_fenomenologicos/v1/metacampos
  umbrales:
    agrupacion_contextos: 0.7
    emergencia_yo: 0.6
    creacion_concepto: 0.5
  neo4j:
    enabled: false
    uri: bolt://localhost:7687
    user: neo4j
    password: password
"""
with open('configuracion/config.yaml', 'w', encoding='utf-8') as f:
    f.write(config_content)
print("config.yaml creado.")

# --- Archivo: scripts/modelos/ontosistema.py ---
ontosistema_py_content = """
import os
import yaml
import datetime
import json

class Ontosistema:
    def __init__(self, config):
        self.config = config
        self.rutas = config.get('modelo_semantico', {}).get('rutas', {})
        self.estadisticas = {
            "fenomenos": 0,
            "contextos": 0,
            "macrocontextos": 0,
            "redes": 0,
            "conceptos_emergentes": 0,
            "apariciones_yo": 0
        }
        self.ultima_actualizacion = datetime.datetime.now().isoformat()

    def actualizar_estadisticas(self):
        for tipo, ruta_relativa in self.rutas.items():
            # En Colab, las rutas son relativas al directorio raíz del notebook
            ruta_completa = ruta_relativa 
            if os.path.exists(ruta_completa):
                archivos = [f for f in os.listdir(ruta_completa) if f.endswith('.yaml')]
                self.estadisticas[tipo] = len(archivos)
            else:
                # Si la ruta no existe, podría ser la primera ejecución, inicializar a 0
                self.estadisticas[tipo] = 0
                # print(f"Advertencia: La ruta {ruta_completa} para {tipo} no existe.") # Opcional: para depuración

        self.contar_apariciones_yo()
        self.ultima_actualizacion = datetime.datetime.now().isoformat()
        return self.estadisticas

    def contar_apariciones_yo(self):
        ruta_contextos_relativa = self.rutas.get('contextos', '')
        # En Colab, las rutas son relativas al directorio raíz del notebook
        ruta_contextos = ruta_contextos_relativa 
        if os.path.exists(ruta_contextos):
            contador = 0
            for archivo in os.listdir(ruta_contextos):
                if archivo.endswith('.yaml'):
                    try:
                        with open(os.path.join(ruta_contextos, archivo), 'r', encoding='utf-8') as f:
                            data = yaml.safe_load(f)
                            if data.get('yo_presente', False):
                                contador += 1
                    except Exception:
                        pass # Ignorar errores de lectura de archivos individuales
            self.estadisticas["apariciones_yo"] = contador
        else:
            self.estadisticas["apariciones_yo"] = 0
            # print(f"Advertencia: La ruta de contextos {ruta_contextos} no existe.") # Opcional

    def guardar_estadisticas(self, ruta_logs):
        os.makedirs(ruta_logs, exist_ok=True) # Asegurar que el directorio de logs exista
        ruta_completa = os.path.join(ruta_logs, 'metricas.json')
        datos = {
            "estadisticas": self.estadisticas,
            "timestamp": self.ultima_actualizacion
        }
        with open(ruta_completa, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)
        return ruta_completa

    def cargar_estadisticas(self, ruta_logs):
        ruta_completa = os.path.join(ruta_logs, 'metricas.json')
        if os.path.exists(ruta_completa):
            with open(ruta_completa, 'r', encoding='utf-8') as f:
                datos = json.load(f)
                self.estadisticas = datos.get("estadisticas", self.estadisticas)
                self.ultima_actualizacion = datos.get("timestamp", self.ultima_actualizacion)
        return self.estadisticas
"""
with open('scripts/modelos/ontosistema.py', 'w', encoding='utf-8') as f:
    f.write(ontosistema_py_content)
print("scripts/modelos/ontosistema.py creado.")

# --- Archivo: scripts/modelos/fenomeno.py (Ejemplo básico, necesitarás el tuyo) ---
fenomeno_py_content = """
import uuid
import datetime

class Fenomeno:
    def __init__(self, contenido, tipo="general", propiedades=None):
        self.id = str(uuid.uuid4())
        self.contenido = contenido
        self.tipo = tipo
        self.propiedades = propiedades or {}
        self.timestamp = datetime.datetime.now().isoformat()

    def to_dict(self):
        return self.__dict__
"""
with open('scripts/modelos/fenomeno.py', 'w', encoding='utf-8') as f:
    f.write(fenomeno_py_content)
print("scripts/modelos/fenomeno.py creado.")

# --- Archivo: scripts/modelos/contexto.py (Ejemplo básico) ---
contexto_py_content = """
import uuid
import datetime
import yaml
import os

class Contexto:
    def __init__(self, descripcion=""):
        self.id = str(uuid.uuid4())
        self.descripcion = descripcion
        self.fenomenos_ids = []
        self.flujos = []
        self.yo_presente = False
        self.proyeccion = ""
        self.timestamp = datetime.datetime.now().isoformat()

    def agregar_fenomeno(self, fenomeno_obj):
        if hasattr(fenomeno_obj, 'id'):
            self.fenomenos_ids.append(fenomeno_obj.id)
        else: # Asumir que es un ID si no es un objeto
            self.fenomenos_ids.append(fenomeno_obj)

    def agregar_flujo(self, flujo_descripcion):
        self.flujos.append(flujo_descripcion)

    def activar_yo(self):
        self.yo_presente = True

    def establecer_proyeccion(self, proyeccion_texto):
        self.proyeccion = proyeccion_texto

    def to_dict(self):
        return self.__dict__

    def guardar(self, ruta_base):
        os.makedirs(ruta_base, exist_ok=True)
        ruta_completa = os.path.join(ruta_base, f"contexto_{self.id}.yaml")
        with open(ruta_completa, 'w', encoding='utf-8') as f:
            yaml.dump(self.to_dict(), f, default_flow_style=False, allow_unicode=True)
        return ruta_completa
"""
with open('scripts/modelos/contexto.py', 'w', encoding='utf-8') as f:
    f.write(contexto_py_content)
print("scripts/modelos/contexto.py creado.")

# --- Archivo: scripts/modelos/macrocontexto.py (Ejemplo básico) ---
macrocontexto_py_content = """
import uuid
import datetime
import yaml
import os

class Macrocontexto:
    def __init__(self, nombre, descripcion=""):
        self.id = str(uuid.uuid4())
        self.nombre = nombre
        self.descripcion = descripcion
        self.contextos_ids = []
        self.patron = ""
        self.reflexion = ""
        self.timestamp = datetime.datetime.now().isoformat()

    def agregar_contexto(self, contexto_id):
        self.contextos_ids.append(contexto_id)

    def establecer_patron(self, patron_texto):
        self.patron = patron_texto

    def establecer_reflexion(self, reflexion_texto):
        self.reflexion = reflexion_texto

    def to_dict(self):
        return self.__dict__

    def guardar(self, ruta_base):
        os.makedirs(ruta_base, exist_ok=True)
        ruta_completa = os.path.join(ruta_base, f"macrocontexto_{self.id}.yaml")
        with open(ruta_completa, 'w', encoding='utf-8') as f:
            yaml.dump(self.to_dict(), f, default_flow_style=False, allow_unicode=True)
        return ruta_completa
"""
with open('scripts/modelos/macrocontexto.py', 'w', encoding='utf-8') as f:
    f.write(macrocontexto_py_content)
print("scripts/modelos/macrocontexto.py creado.")

# --- Archivo: scripts/modelos/red_contextos.py (Ejemplo básico) ---
red_contextos_py_content = """
import uuid
import datetime

class RedContextos:
    def __init__(self, nombre="Red por defecto"):
        self.id = str(uuid.uuid4())
        self.nombre = nombre
        self.nodos = [] # IDs de contextos
        self.aristas = [] # Tuplas (id_origen, id_destino, tipo_relacion)
        self.timestamp = datetime.datetime.now().isoformat()

    def to_dict(self):
        return self.__dict__
"""
with open('scripts/modelos/red_contextos.py', 'w', encoding='utf-8') as f:
    f.write(red_contextos_py_content)
print("scripts/modelos/red_contextos.py creado.")

# --- Archivo: scripts/modelos/metacampo.py (Ejemplo básico) ---
metacampo_py_content = """
import uuid
import datetime

class Metacampo:
    def __init__(self, nombre, descripcion=""):
        self.id = str(uuid.uuid4())
        self.nombre = nombre
        self.descripcion = descripcion
        self.conceptos_asociados = [] # IDs de conceptos
        self.timestamp = datetime.datetime.now().isoformat()

    def to_dict(self):
        return self.__dict__
"""
with open('scripts/modelos/metacampo.py', 'w', encoding='utf-8') as f:
    f.write(metacampo_py_content)
print("scripts/modelos/metacampo.py creado.")

# --- Archivo: scripts/extractor_features.py ---
extractor_features_py_content = """
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import yaml
import os
import nltk
from nltk.corpus import stopwords

# Descargar stopwords si no están presentes (solo la primera vez)
try:
    stopwords.words('spanish')
except LookupError:
    nltk.download('stopwords')

def cargar_config_extractor(): # Renombrado para evitar colisión
    # En Colab, la ruta es relativa al directorio raíz del notebook
    ruta = 'configuracion/config.yaml'
    with open(ruta, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    return config

class ExtractorFeatures:
    def __init__(self, config=None):
        self.config = config if config else cargar_config_extractor()
        self.vectorizer = TfidfVectorizer(
            max_features=self.config['procesamiento']['max_features_tfidf'],
            ngram_range=tuple(self.config['procesamiento']['ngram_range']),
            stop_words=stopwords.words(self.config['procesamiento']['idioma']) if self.config['procesamiento']['eliminar_stopwords'] else None
        )

    def limpiar_texto(self, texto):
        # Implementa tu lógica de limpieza aquí si es necesario
        return texto

    def extraer_features_tfidf(self, textos_crudos):
        textos_limpios = [self.limpiar_texto(texto) for texto in textos_crudos]
        if not textos_limpios: # Si no hay textos, devuelve un DataFrame vacío
            return pd.DataFrame()
        features = self.vectorizer.fit_transform(textos_limpios)
        df_features = pd.DataFrame(features.toarray(), columns=self.vectorizer.get_feature_names_out())
        return df_features

if __name__ == "__main__":
    # Ejemplo de uso (adaptado para Colab)
    config_main = cargar_config_extractor()
    extractor = ExtractorFeatures(config_main)
    
    # Crear archivos de ejemplo en entrada_bruta para la prueba
    os.makedirs('entrada_bruta', exist_ok=True)
    textos_ejemplo = [
        "Este es el primer texto fenomenológico sobre la experiencia vivida.",
        "Un segundo relato acerca de la conciencia temporal y el ser.",
        "La corporalidad se manifiesta en cada movimiento y sensación.",
        "Explorando la mundaneidad de los objetos cotidianos.",
        "Otro texto más para tener suficientes datos.",
        "La percepción del tiempo es fundamental.",
        "Siento mi cuerpo en el espacio.",
        "Los objetos me rodean en mi mundo."
    ]
    for i, texto in enumerate(textos_ejemplo):
        with open(f'entrada_bruta/texto_fenomenologico_{i+1}.txt', 'w', encoding='utf-8') as f:
            f.write(texto)

    # Cargar textos desde los archivos creados
    textos_cargados = []
    for i in range(1, 9):
        with open(f'entrada_bruta/texto_fenomenologico_{i}.txt', 'r', encoding='utf-8') as f:
            textos_cargados.append(f.read())

    if textos_cargados:
        df_features = extractor.extraer_features_tfidf(textos_cargados)
        print("Características extraídas:")
        print(df_features.head())
        
        # Guardar features y etiquetas (ejemplo)
        os.makedirs('features_extraidas', exist_ok=True)
        df_features.to_csv('features_extraidas/features_tfidf.csv', index=False)
        print("features_tfidf.csv guardado.")
        
        # Crear archivo de etiquetas de ejemplo
        etiquetas_ejemplo = [
            'experiencia_vivida',
            'conciencia_temporal',
            'corporalidad',
            'mundaneidad',
            'experiencia_vivida', # Asegurar al menos 2 por clase para test_size=0.5
            'conciencia_temporal',
            'corporalidad',
            'mundaneidad'
        ]
        df_etiquetas = pd.DataFrame(etiquetas_ejemplo, columns=['clase'])
        df_etiquetas.to_csv('features_extraidas/etiquetas.csv', index=False)
        print("etiquetas.csv guardado.")
    else:
        print("No se cargaron textos para extraer características.")
"""
with open('scripts/extractor_features.py', 'w', encoding='utf-8') as f:
    f.write(extractor_features_py_content)
print("scripts/extractor_features.py creado.")

# --- Archivo: scripts/clasificador.py ---
clasificador_py_content = """
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import yaml
import os

def cargar_config_clasificador(): # Renombrado para evitar colisión
    ruta = 'configuracion/config.yaml'
    with open(ruta, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    return config

class ClasificadorFenomenologico:
    def __init__(self, config):
        self.config = config
        self.modelo = RandomForestClassifier(
            n_estimators=config['clasificacion']['n_estimators'],
            random_state=config['clasificacion']['random_state'],
            n_jobs=-1 # Usar todos los procesadores disponibles
        )

    def entrenar(self, X, y):
        if len(X) == 0 or len(y) == 0:
            print("No hay datos para entrenar.")
            return
        if len(X) != len(y):
            raise ValueError(f"El número de muestras no coincide: {len(X)} features vs {len(y)} etiquetas")
        
        # Verificar si hay suficientes muestras por clase para stratify
        from collections import Counter
        conteo_clases = Counter(y)
        clases_con_una_muestra = [clase for clase, conteo in conteo_clases.items() if conteo < 2]
        
        stratify_param = y
        if clases_con_una_muestra and self.config['clasificacion']['test_size'] > 0:
            print(f"Advertencia: Las siguientes clases tienen solo 1 muestra y test_size > 0: {clases_con_una_muestra}. No se usará 'stratify'.")
            stratify_param = None
            
        # Asegurar que test_size no sea mayor que el número de muestras de la clase más pequeña si stratify está activo
        min_samples_in_class = min(conteo_clases.values()) if conteo_clases else 0
        current_test_size = self.config['clasificacion']['test_size']

        if stratify_param is not None and min_samples_in_class > 0 and int(min_samples_in_class * current_test_size) < 1:
            # Si el test_size resultaría en 0 muestras de prueba para la clase más pequeña, ajustar o advertir
            # Por simplicidad, aquí simplemente no usamos stratify si podría causar problemas.
            # Una mejor solución sería ajustar test_size o manejarlo de forma más robusta.
            print(f"Advertencia: test_size ({current_test_size}) es demasiado grande para la clase más pequeña ({min_samples_in_class} muestras). No se usará 'stratify'.")
            stratify_param = None

        if len(set(y)) < 2 and current_test_size > 0:
             print("Advertencia: Solo hay una clase presente. El conjunto de prueba estará vacío o el entrenamiento fallará si test_size > 0.")
             # No se puede dividir si solo hay una clase y test_size > 0
             X_train, X_test, y_train, y_test = X, pd.DataFrame(), y, [] # O manejar de otra forma
        elif current_test_size == 0: # Si test_size es 0, usar todos los datos para entrenar
            X_train, X_test, y_train, y_test = X, pd.DataFrame(), y, []
        else:
            try:
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y,
                    test_size=current_test_size,
                    random_state=self.config['clasificacion']['random_state'],
                    stratify=stratify_param
                )
            except ValueError as e:
                 print(f"Error durante train_test_split (probablemente debido a pocas muestras por clase con stratify): {e}. Intentando sin stratify.")
                 X_train, X_test, y_train, y_test = train_test_split(
                    X, y,
                    test_size=current_test_size,
                    random_state=self.config['clasificacion']['random_state'],
                    stratify=None # Intentar sin stratify
                )

        if not X_train.empty if isinstance(X_train, pd.DataFrame) else len(X_train) > 0:
            self.modelo.fit(X_train, y_train)
            if not X_test.empty if isinstance(X_test, pd.DataFrame) else len(X_test) > 0:
                score = self.modelo.score(X_test, y_test)
                print(f"Precisión en test: {score:.2f}")
            else:
                print("El conjunto de test está vacío, no se calcula la precisión.")
        else:
            print("El conjunto de entrenamiento está vacío.")

    def predecir(self, X):
        if not hasattr(self.modelo, 'classes_'):
            print("El modelo no ha sido entrenado aún.")
            return []
        return self.modelo.predict(X)

    def predecir_probabilidades(self, X):
        if not hasattr(self.modelo, 'classes_'):
            print("El modelo no ha sido entrenado aún.")
            return []
        return self.modelo.predict_proba(X)

if __name__ == "__main__":
    config_main = cargar_config_clasificador()
    
    # Cargar features (asegúrate que extractor_features.py se haya ejecutado)
    features_path = 'features_extraidas/features_tfidf.csv'
    etiquetas_path = 'features_extraidas/etiquetas.csv'

    if os.path.exists(features_path) and os.path.exists(etiquetas_path):
        df = pd.read_csv(features_path)
        etiquetas_df = pd.read_csv(etiquetas_path)
        y = etiquetas_df['clase'].tolist()
        
        if len(df) != len(y):
            raise ValueError(f"Número de muestras no coincide: {len(df)} features vs {len(y)} etiquetas")
        
        if df.empty:
            print("El DataFrame de características está vacío.")
        else:
            clf = ClasificadorFenomenologico(config_main)
            clf.entrenar(df, y) # df ya son los valores numéricos
            # Ejemplo de predicción
            if hasattr(clf.modelo, 'classes_'): # Verificar si el modelo fue entrenado
                 predicciones = clf.predecir(df)
                 print(f"Predicciones de ejemplo: {predicciones[:5]}")
                 probabilidades = clf.predecir_probabilidades(df)
                 print(f"Probabilidades de ejemplo: {probabilidades[:5]}")
    else:
        print(f"Asegúrate que '{features_path}' y '{etiquetas_path}' existan. Ejecuta extractor_features.py primero.")
"""
with open('scripts/clasificador.py', 'w', encoding='utf-8') as f:
    f.write(clasificador_py_content)
print("scripts/clasificador.py creado.")

# --- Archivo: scripts/gestor_modelo_semantico.py ---
gestor_modelo_semantico_py_content = """
import os
import yaml
import datetime
# Asegúrate que las rutas de importación sean correctas para Colab
from scripts.modelos.fenomeno import Fenomeno
from scripts.modelos.contexto import Contexto
from scripts.modelos.macrocontexto import Macrocontexto
from scripts.modelos.red_contextos import RedContextos
from scripts.modelos.ontosistema import Ontosistema
from scripts.modelos.metacampo import Metacampo

def cargar_config_gestor(): # Renombrado
    ruta = 'configuracion/config.yaml'
    with open(ruta, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    return config

class GestorModeloSemantico:
    def __init__(self, config=None):
        self.config = config or cargar_config_gestor()
        self.inicializar_config_modelo() # Asegura que la sección 'modelo_semantico' exista
        self.ontosistema = Ontosistema(self.config)
        self.crear_directorios_modelo() # Renombrado para claridad
    
    def inicializar_config_modelo(self):
        if 'modelo_semantico' not in self.config:
            self.config['modelo_semantico'] = {}
        if 'rutas' not in self.config['modelo_semantico']:
            self.config['modelo_semantico']['rutas'] = {
                'fenomenos': 'procesado/nodos_fenomenologicos/v1/fenomenos',
                'contextos': 'procesado/nodos_fenomenologicos/v1/contextos',
                'macrocontextos': 'procesado/nodos_fenomenologicos/v1/macrocontextos',
                'redes': 'procesado/nodos_fenomenologicos/v1/redes',
                'conceptos_emergentes': 'procesado/nodos_fenomenologicos/v1/conceptos',
                'metacampos': 'procesado/nodos_fenomenologicos/v1/metacampos'
            }
        # Podrías añadir más inicializaciones de umbrales, neo4j, etc. si es necesario
        # self.guardar_config_gestor() # Opcional: guardar si se modificó

    def guardar_config_gestor(self): # Renombrado
        ruta = 'configuracion/config.yaml'
        with open(ruta, 'w', encoding='utf-8') as f:
            yaml.dump(self.config, f, default_flow_style=False, allow_unicode=True)

    def crear_directorios_modelo(self): # Renombrado
        if 'rutas' in self.config.get('modelo_semantico', {}):
            for tipo, ruta_relativa in self.config['modelo_semantico']['rutas'].items():
                # En Colab, las rutas son relativas al directorio raíz del notebook
                os.makedirs(ruta_relativa, exist_ok=True)
        else:
            print("Advertencia: No se encontraron rutas en la configuración del modelo semántico para crear directorios.")

    def crear_fenomeno(self, contenido, tipo="general", propiedades=None):
        fenomeno = Fenomeno(contenido, tipo, propiedades)
        ruta_base_relativa = self.config['modelo_semantico']['rutas']['fenomenos']
        # ruta_base = os.path.join(os.path.dirname(__file__), '..', ruta_base_relativa) # Ajuste para Colab
        ruta_base = ruta_base_relativa
        os.makedirs(ruta_base, exist_ok=True)
        ruta_completa = os.path.join(ruta_base, f"fenomeno_{fenomeno.id}.yaml")
        with open(ruta_completa, 'w', encoding='utf-8') as f:
            yaml.dump(fenomeno.to_dict(), f, default_flow_style=False, allow_unicode=True)
        return fenomeno

    def crear_contexto(self, descripcion="", fenomenos=None, flujos=None, yo_presente=False, proyeccion=""):
        contexto = Contexto(descripcion)
        if fenomenos: [contexto.agregar_fenomeno(f) for f in fenomenos]
        if flujos: [contexto.agregar_flujo(flujo) for flujo in flujos]
        if yo_presente: contexto.activar_yo()
        if proyeccion: contexto.establecer_proyeccion(proyeccion)
        
        ruta_base_relativa = self.config['modelo_semantico']['rutas']['contextos']
        # ruta_base = os.path.join(os.path.dirname(__file__), '..', ruta_base_relativa) # Ajuste para Colab
        ruta_base = ruta_base_relativa
        contexto.guardar(ruta_base)
        return contexto

    def crear_macrocontexto(self, nombre, descripcion, contextos_ids, patron="", reflexion=""):
        macrocontexto = Macrocontexto(nombre, descripcion)
        for ctx_id in contextos_ids: macrocontexto.agregar_contexto(ctx_id)
        if patron: macrocontexto.establecer_patron(patron)
        if reflexion: macrocontexto.establecer_reflexion(reflexion)
        
        ruta_base_relativa = self.config['modelo_semantico']['rutas']['macrocontextos']
        # ruta_base = os.path.join(os.path.dirname(__file__), '..', ruta_base_relativa) # Ajuste para Colab
        ruta_base = ruta_base_relativa
        macrocontexto.guardar(ruta_base)
        return macrocontexto

    def actualizar_estadisticas(self):
        self.ontosistema.actualizar_estadisticas()
        # ruta_logs = os.path.join(os.path.dirname(__file__), '..', 'logs_sistema') # Ajuste para Colab
        ruta_logs = 'logs_sistema'
        self.ontosistema.guardar_estadisticas(ruta_logs)
        return self.ontosistema.estadisticas
    
    def obtener_estado(self): # Método añadido en analizador_local.py
        # Simula la obtención de un estado, puedes hacerlo más complejo
        stats = self.ontosistema.estadisticas
        return {
            "total_conceptos": stats.get("conceptos_emergentes", 0) + stats.get("fenomenos", 0), # Ejemplo
            "total_relaciones": stats.get("contextos", 0) + stats.get("redes",0) # Ejemplo
        }

if __name__ == "__main__":
    gestor = GestorModeloSemantico()
    # gestor.crear_directorios_modelo() # Ya se llama en __init__
    
    fenomeno1 = gestor.crear_fenomeno("Sensación de Colab", "digital")
    contexto1 = gestor.crear_contexto("Usando Colab", fenomenos=[fenomeno1], yo_presente=True)
    
    stats = gestor.actualizar_estadisticas()
    print("Estadísticas del gestor actualizadas:")
    for k, v in stats.items():
        print(f"  - {k}: {v}")
"""
with open('scripts/gestor_modelo_semantico.py', 'w', encoding='utf-8') as f:
    f.write(gestor_modelo_semantico_py_content)
print("scripts/gestor_modelo_semantico.py creado.")

# --- Archivo: scripts/analizador_sistema.py ---
analizador_sistema_py_content = """
import os
import pandas as pd
import yaml # Asegurar importación
from collections import Counter # Asegurar importación

# Ajustar rutas de importación para Colab
from scripts.clasificador import ClasificadorFenomenologico #, cargar_config_clasificador (usar una sola func cargar_config)
from scripts.gestor_modelo_semantico import GestorModeloSemantico

# Función unificada para cargar config, ya que todas las versiones son iguales
def cargar_config_analizador(): 
    ruta = 'configuracion/config.yaml'
    with open(ruta, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    return config

def analizar_sistema():
    # ruta_base_script = os.path.dirname(__file__) # No aplicable directamente en Colab para __file__
    # reporte_path = os.path.join(ruta_base_script, '..', 'logs_sistema', 'reporte_analisis.txt')
    reporte_path = 'logs_sistema/reporte_analisis.txt'
    os.makedirs('logs_sistema', exist_ok=True)

    with open(reporte_path, 'w', encoding='utf-8') as f:
        f.write("=== REPORTE DE ANÁLISIS DEL SISTEMA (COLAB) ===\n\n")
        
        directorios = ['entrada_bruta', 'features_extraidas', 'logs_sistema'] # 'clasificados' no se usa activamente aquí
        f.write("1. Verificación de directorios:\n")
        for dir_name in directorios:
            # path = os.path.join(ruta_base_script, '..', dir_name)
            path = dir_name # Rutas relativas en Colab
            existe = os.path.exists(path)
            f.write(f"   - {dir_name}: {'✓' if existe else 'X'}\n")
        
        f.write("\n2. Verificación de archivos:\n")
        archivos_necesarios = {
            'features_tfidf.csv': 'features_extraidas/features_tfidf.csv',
            'etiquetas.csv': 'features_extraidas/etiquetas.csv',
            'config.yaml': 'configuracion/config.yaml'
        }
        for nombre, ruta_relativa in archivos_necesarios.items():
            # path = os.path.join(ruta_base_script, '..', ruta_relativa)
            path = ruta_relativa # Rutas relativas en Colab
            existe = os.path.exists(path)
            f.write(f"   - {nombre}: {'✓' if existe else 'X'}\n")
        
        try:
            config = cargar_config_analizador()
            
            # Verificar si los archivos de datos existen antes de intentar leerlos
            if not os.path.exists(archivos_necesarios['features_tfidf.csv']):
                f.write(f"\nERROR: El archivo {archivos_necesarios['features_tfidf.csv']} no existe. Ejecuta la celda de extracción de features primero.\n")
                df = pd.DataFrame() # DataFrame vacío para evitar errores posteriores
            else:
                df = pd.read_csv(archivos_necesarios['features_tfidf.csv'])

            if not os.path.exists(archivos_necesarios['etiquetas.csv']):
                f.write(f"\nERROR: El archivo {archivos_necesarios['etiquetas.csv']} no existe. Ejecuta la celda de extracción de features primero.\n")
                y = [] # Lista vacía
            else:
                etiquetas_df = pd.read_csv(archivos_necesarios['etiquetas.csv'])
                y = etiquetas_df['clase'].tolist()
            
            f.write(f"\n3. Análisis de datos:\n")
            f.write(f"   - Número de textos (features): {len(df)}\n")
            f.write(f"   - Número de features: {df.shape[1] if not df.empty else 0}\n")
            f.write(f"   - Número de etiquetas: {len(y)}\n")
            f.write(f"   - Categorías únicas: {len(set(y)) if y else 0}\n")
            
            conteo = Counter(y)
            f.write("\n4. Distribución de categorías:\n")
            for categoria, count in conteo.items():
                f.write(f"   - {categoria}: {count} ejemplos\n")
                if count < 2 and config['clasificacion']['test_size'] > 0:
                    f.write(f"     ¡ADVERTENCIA! La categoría {categoria} tiene menos de 2 ejemplos, puede afectar la división train/test.\n")
            
            f.write("\n5. Análisis del modelo semántico:\n")
            try:
                gestor = GestorModeloSemantico(config)
                if 'modelo_semantico' in config and 'rutas' in config['modelo_semantico']:
                    for nivel, ruta_relativa_ms in config['modelo_semantico']['rutas'].items():
                        # ruta_completa_ms = os.path.join(ruta_base_script, '..', ruta_relativa_ms)
                        ruta_completa_ms = ruta_relativa_ms # Rutas relativas en Colab
                        existe_ms = os.path.exists(ruta_completa_ms)
                        f.write(f"   - Directorio {nivel}: {'✓' if existe_ms else 'X'}\n")
                else:
                    f.write("   - No se encontró la configuración de rutas del modelo semántico.\n")
                
                stats = gestor.actualizar_estadisticas() # Actualiza y obtiene las estadísticas
                f.write("\n6. Estadísticas del modelo semántico:\n")
                for k, v_stat in stats.items(): f.write(f"   - {k.capitalize()}: {v_stat}\n")
                
                f.write("\n7. Análisis de escalabilidad:\n")
                total_elementos = sum(stats.values())
                f.write(f"   - Total de elementos: {total_elementos}\n")
                if total_elementos > 0 and stats.get('contextos', 0) > 0: # Evitar división por cero
                    porcentaje_yo = (stats.get('apariciones_yo',0) / stats['contextos']) * 100 if stats['contextos'] > 0 else 0
                    f.write(f"   - Porcentaje de emergencia del YO (en contextos): {porcentaje_yo:.1f}%\n")
                    if stats.get('redes',0) > 0:
                        densidad = stats['contextos'] / stats['redes']
                        f.write(f"   - Densidad de red (contextos/red): {densidad:.1f}\n")
                else:
                    f.write("   - No hay suficientes elementos para calcular porcentaje de YO o densidad de red.\n")

                f.write("\n8. Verificación de integridad (simplificada para Colab):\n")
                # Esta verificación es más compleja de replicar fielmente sin la estructura de archivos local exacta.
                # Se omite la comprobación detallada de conteo de archivos vs stats para simplificar.
                f.write("   - Integridad de datos: (Verificación detallada omitida en Colab)\n")

            except Exception as e_ms:
                f.write(f"   - ERROR en análisis del modelo semántico: {str(e_ms)}\n")
            
            f.write("\n9. Prueba de clasificación:\n")
            if not df.empty and y:
                try:
                    clasificador = ClasificadorFenomenologico(config)
                    # X_values = df.values # No es necesario si df ya es numérico y sin index/columnas no deseadas
                    
                    if len(set(y)) < 2 and config['clasificacion']['test_size'] > 0:
                        f.write("   - ERROR: Se necesitan al menos 2 categorías diferentes para entrenar con un test_size > 0.\n")
                    elif len(df) < 2 : # O alguna otra heurística para "muy pocos datos"
                         f.write("   - ERROR: Muy pocos datos para realizar una división train/test significativa.\n")
                    else:
                        clasificador.entrenar(df, y) # Pasar el DataFrame directamente
                        f.write("   - Entrenamiento: ✓ (Ver consola para precisión)\n")
                        if hasattr(clasificador.modelo, 'classes_'): # Si el modelo se entrenó
                            prediccion = clasificador.predecir(df.head(1)) # Predecir sobre la primera fila
                            probabilidades = clasificador.predecir_probabilidades(df.head(1))
                            f.write(f"   - Predicción de prueba (1ra muestra): {prediccion[0] if prediccion else 'N/A'}\n")
                            f.write(f"   - Probabilidad máxima (1ra muestra): {max(probabilidades[0]) if probabilidades and probabilidades[0].size > 0 else 'N/A':.3f}\n")
                        else:
                            f.write("   - Modelo no entrenado, no se pueden hacer predicciones.\n")
                except Exception as e_clf:
                    f.write(f"   - ERROR en clasificación: {str(e_clf)}\n")
            else:
                f.write("   - No hay suficientes datos (features o etiquetas) para la prueba de clasificación.\n")

        except FileNotFoundError as e_fnf:
            f.write(f"\nERROR CRÍTICO: No se pudo cargar {e_fnf.filename}. Asegúrate que los archivos base existan.\n")
        except Exception as e_gen:
            f.write(f"\nERROR GENERAL en el análisis: {str(e_gen)}\n")
        
        f.write("\n10. Recomendaciones (Colab):\n")
        f.write("   - Asegúrate de ejecutar las celdas en orden (instalación, creación de archivos, carga de datos, etc.)\n")
        f.write("   - Verifica que 'configuracion/config.yaml' esté correctamente definido en su celda.\n")
        f.write("   - Sube tus archivos .txt a la carpeta 'entrada_bruta' o créalos en una celda.\n")
        f.write("   - Ejecuta la celda de 'extractor_features.py' para generar 'features_tfidf.csv' y 'etiquetas.csv'.\n")
        f.write("   - Revisa los mensajes de 'Precisión en test' en la salida de la celda de entrenamiento del clasificador.\n")
        
        f.write(f"\n=== FIN DEL REPORTE (COLAB) ===\n")
    
    print(f"Reporte generado en: {reporte_path}")
    return reporte_path

if __name__ == "__main__":
    # Para ejecutar este script directamente en Colab (después de crear los otros archivos):
    # Primero, asegúrate que extractor_features.py haya creado los CSV necesarios.
    # Puedes simular la ejecución de extractor_features.py aquí si es necesario para la prueba:
    print("Ejecutando extractor_features.py para asegurar datos...")
    exec(open('scripts/extractor_features.py').read())
    print("Ejecutando clasificador.py para asegurar modelo...")
    exec(open('scripts/clasificador.py').read()) # Para que el modelo se entrene si es necesario
    print("Ejecutando gestor_modelo_semantico.py para asegurar estructuras...")
    exec(open('scripts/gestor_modelo_semantico.py').read())

    print("\nIniciando análisis del sistema...")
    analizar_sistema()
    print("Análisis completado. Revisa el archivo 'reporte_analisis.txt' en la carpeta logs_sistema.")
"""
with open('scripts/analizador_sistema.py', 'w', encoding='utf-8') as f:
    f.write(analizador_sistema_py_content)
print("scripts/analizador_sistema.py creado.")

print("Todos los archivos .py y config.yaml han sido creados/escritos.")

In [ ]:
# Ejemplo: Crear archivos de texto de entrada si no los subes manualmente
textos_fenomenologicos = {
    "texto_fenomenologico_1.txt": "Primera experiencia sobre la percepción del espacio y la luz.",
    "texto_fenomenologico_2.txt": "Recuerdo vívido de una conversación y las emociones asociadas.",
    "texto_fenomenologico_3.txt": "La temporalidad se siente diferente en momentos de alta concentración.",
    "texto_fenomenologico_4.txt": "Mi cuerpo reacciona al frío de la mañana.",
    "texto_fenomenologico_5.txt": "Observo el mundo a través de la ventana, los objetos parecen lejanos.",
    "texto_fenomenologico_6.txt": "La intersubjetividad se manifiesta en la mirada del otro.",
    "texto_fenomenologico_7.txt": "Una reflexión sobre la intencionalidad de mis acciones pasadas.",
    "texto_fenomenologico_8.txt": "La afectividad tiñe todos mis recuerdos de la infancia."
}

for nombre_archivo, contenido_texto in textos_fenomenologicos.items():
    with open(os.path.join('entrada_bruta', nombre_archivo), 'w', encoding='utf-8') as f:
        f.write(contenido_texto)

print(f"{len(textos_fenomenologicos)} archivos de texto creados en 'entrada_bruta'.")

In [ ]:
import pandas as pd
import numpy as np
import yaml
import os
import sys
import datetime # Asegurar que datetime esté importado
from typing import Dict, List # Para type hints si los usas en tus clases

# Agregar el directorio scripts al path para que Python encuentre los módulos
# Esto es crucial en Colab si tus scripts están en un subdirectorio.
if 'scripts' not in sys.path:
    sys.path.append('scripts')
if '.' not in sys.path: # Asegurar que el directorio actual también esté en el path
    sys.path.append('.')

# Ahora intenta importar tus módulos
try:
    from scripts.extractor_features import ExtractorFeatures, cargar_config_extractor
    from scripts.clasificador import ClasificadorFenomenologico, cargar_config_clasificador
    from scripts.gestor_modelo_semantico import GestorModeloSemantico, cargar_config_gestor
    from scripts.analizador_sistema import analizar_sistema, cargar_config_analizador
    # Si tienes clases en sistema_principal_v2.py que necesitas, impórtalas también
    # from sistema_principal_v2 import SistemaFenomenologicoV2 # Ejemplo
    print("Módulos importados correctamente.")
except ModuleNotFoundError as e:
    print(f"Error al importar módulos: {e}")
    print("Asegúrate que los archivos .py estén en el directorio 'scripts' y que la Celda 3 se haya ejecutado completamente.")
except ImportError as e:
    print(f"Error de importación: {e}")
    print("Verifica las dependencias internas entre tus scripts.")

# Cargar configuración (usando una de las funciones, son equivalentes)
config_global = cargar_config_analizador() 

In [ ]:
print("Ejecutando extractor_features.py para generar archivos CSV...")
# Usamos exec() para correr el script como si fuera desde la línea de comandos
# Esto ejecutará el bloque if __name__ == "__main__": de extractor_features.py
try:
    # Leer el contenido del script
    with open('scripts/extractor_features.py', 'r', encoding='utf-8') as f:
        script_content = f.read()
    # Ejecutar el script en el contexto global actual
    exec(script_content, globals())
    print("Extractor de features ejecutado.")
except FileNotFoundError:
    print("Error: scripts/extractor_features.py no encontrado. Asegúrate que la Celda 3 se ejecutó.")
except Exception as e:
    print(f"Error al ejecutar extractor_features.py: {e}")

# Verificar la creación de archivos
if os.path.exists('features_extraidas/features_tfidf.csv') and os.path.exists('features_extraidas/etiquetas.csv'):
    print("Archivos features_tfidf.csv y etiquetas.csv generados correctamente.")
    df_check = pd.read_csv('features_extraidas/features_tfidf.csv')
    et_check = pd.read_csv('features_extraidas/etiquetas.csv')
    print(f"features_tfidf.csv tiene {df_check.shape[0]} filas y {df_check.shape[1]} columnas.")
    print(f"etiquetas.csv tiene {et_check.shape[0]} filas.")
else:
    print("Error: Los archivos features_tfidf.csv o etiquetas.csv no fueron generados.")

In [ ]:
print("\nEjecutando clasificador.py para entrenar el modelo...")
try:
    with open('scripts/clasificador.py', 'r', encoding='utf-8') as f:
        script_content_clf = f.read()
    exec(script_content_clf, globals())
    print("Clasificador ejecutado y modelo entrenado (si había datos suficientes).")
except FileNotFoundError:
    print("Error: scripts/clasificador.py no encontrado.")
except Exception as e:
    print(f"Error al ejecutar clasificador.py: {e}")

In [ ]:
print("\nEjecutando gestor_modelo_semantico.py para inicializar estructuras...")
try:
    with open('scripts/gestor_modelo_semantico.py', 'r', encoding='utf-8') as f:
        script_content_gms = f.read()
    exec(script_content_gms, globals())
    print("Gestor del modelo semántico ejecutado.")
except FileNotFoundError:
    print("Error: scripts/gestor_modelo_semantico.py no encontrado.")
except Exception as e:
    print(f"Error al ejecutar gestor_modelo_semantico.py: {e}")

In [ ]:
print("\nEjecutando el análisis completo del sistema...")
try:
    # La función analizar_sistema ya está importada
    ruta_reporte = analizar_sistema() # Llama a la función directamente
    print(f"Análisis del sistema completado. Reporte en: {ruta_reporte}")
    
    # Mostrar el contenido del reporte
    if os.path.exists(ruta_reporte):
        with open(ruta_reporte, 'r', encoding='utf-8') as f_report:
            print("\n--- Contenido del Reporte ---")
            print(f_report.read())
            print("--- Fin del Reporte ---")
    else:
        print(f"No se pudo encontrar el archivo de reporte en {ruta_reporte}")
        
except NameError as ne:
    print(f"Error de nombre: {ne}. Asegúrate que la función 'analizar_sistema' esté correctamente importada y definida.")
    print("Verifica la Celda 5 (Importaciones).")
except Exception as e:
    print(f"Ocurrió un error durante el análisis del sistema: {e}")